## 数据清理

In [12]:
import pandas as pd
import re

# 定义数据清理函数
def clean_text(text):
    if isinstance(text, str):  # 确保文本是字符串
        text = re.sub(r'[^\w\s]', '', text)  # 去除特殊符号
        text = re.sub(r'\s+', ' ', text)     # 替换多个空格为一个空格
        return text.strip()                   # 去除首尾空格
    return ''  # 返回空字符串作为默认值

# 1. 清理 fengchuoibanxia_comments_all_translated.xlsx
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_all_translated.xlsx')

# 去除重复评论
fengchuoibanxia_df.drop_duplicates(inplace=True)

# 去除无关字符
fengchuoibanxia_df['Comment'] = fengchuoibanxia_df['Comment'].apply(clean_text)

# 处理空值，选择填充空字符串
fengchuoibanxia_df['Comment'].fillna('', inplace=True)

# 保存清理后的数据
fengchuoibanxia_df.to_excel('fengchuoibanxia_comments_cleaned.xlsx', index=False)

# 2. 清理 shanhaiqing_comments_all_translated.xlsx
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_all_translated.xlsx')

# 去除重复评论
shanhaiqing_df.drop_duplicates(inplace=True)

# 去除无关字符
shanhaiqing_df['Comment'] = shanhaiqing_df['Comment'].apply(clean_text)

# 处理空值，选择填充空字符串
shanhaiqing_df['Comment'].fillna('', inplace=True)

# 保存清理后的数据
shanhaiqing_df.to_excel('shanhaiqing_comments_cleaned.xlsx', index=False)

print("数据清理完成，已保存为 fengchuoibanxia_comments_cleaned.xlsx 和 shanhaiqing_comments_cleaned.xlsx")


数据清理完成，已保存为 fengchuoibanxia_comments_cleaned.xlsx 和 shanhaiqing_comments_cleaned.xlsx


In [13]:
import pandas as pd

# 加载清理后的数据
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_cleaned.xlsx')
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_cleaned.xlsx')


In [22]:
import pandas as pd
import jieba
from collections import Counter
from nltk import ngrams

# 加载清理后的数据
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_cleaned.xlsx')
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_cleaned.xlsx')

# 自定义停用词列表
stop_words = set([
    '的', '了', '是', '我', '都', '有', '好', '也', '不', '在', 
    '与', '和', '这', '那', '要', '就', '人', '你', '说', '而', '对', '着', '如', '于'
])

# 定义一个函数进行词频分析
def analyze_word_frequency(df, title):
    # 分词并去除停用词，处理缺失值
    df['分词'] = df['Comment'].apply(lambda x: [word for word in jieba.cut(x) if word not in stop_words] if isinstance(x, str) else [])

    # 提取2-grams和3-grams
    n_grams_2 = []
    n_grams_3 = []
    for comment in df['分词']:
        n_grams_2.extend(ngrams(comment, 2))  # 提取双词组
        n_grams_3.extend(ngrams(comment, 3))  # 提取三词组

    # 统计词频
    word_counts_2 = Counter(n_grams_2)
    word_counts_3 = Counter(n_grams_3)

    # 提取Top 10词组
    top_10_words_2 = word_counts_2.most_common(10)
    top_10_words_3 = word_counts_3.most_common(10)

    # 输出结果
    print(f"{title} 的 Top 10二字词组及其出现次数：", {''.join(word): count for word, count in top_10_words_2})
    print(f"{title} 的 Top 10三字词组及其出现次数：", {''.join(word): count for word, count in top_10_words_3})

# 分析 fengchuoibanxia 数据集
analyze_word_frequency(fengchuoibanxia_df, "风吹半夏")

# 分析 shanhaiqing 数据集
analyze_word_frequency(shanhaiqing_df, "山海情")


风吹半夏 的 Top 10二字词组及其出现次数： {'许半夏': 26, '这部剧': 19, '小陈': 13, '风吹半夏': 9, '太好看': 9, '伍建设': 9, ' 赵丽颖': 8, '90年代': 7, '李光洁': 7, '啊 ': 7}
风吹半夏 的 Top 10三字词组及其出现次数： {'第一次看赵丽颖': 3, '什么像什么': 3, '看这部剧': 2, '趙麗穎演技': 2, '上千万生意': 2, '幸福到万家': 2, '怎么没有字幕': 2, '看赵丽颖电视剧': 2, '喜欢李光洁': 2, '為什麼沒有': 2}
山海情 的 Top 10二字词组及其出现次数： {'凌教授': 78, '白老师': 58, '接地气': 44, '这部剧': 43, '莫小贝': 42, '看哭': 40, '大江大河': 38, '啊 ': 37, '太好看': 35, '这个剧': 35}
山海情 的 Top 10三字词组及其出现次数： {'看这个剧': 11, '大江大河2': 11, '太接地气': 10, '把看哭': 9, '佟掌柜郭': 9, '掌柜郭芙蓉': 9, '哈哈哈哈哈哈哈哈哈': 9, '越看越': 7, '琅琊榜': 7, '过上好日子': 6}


In [ ]:
# 定义一个函数进行词频分析
def analyze_word_frequency_top(df, title):
    # 分词并去除停用词，处理缺失值
    df['分词'] = df['Comment'].apply(lambda x: [word for word in jieba.cut(x) if word not in stop_words] if isinstance(x, str) else [])

    # 统计词频
    all_words = [word for comment in df['分词'] for word in comment]
    word_counts = Counter(all_words)

    # 提取Top 10词
    top_10_words = word_counts.most_common(10)
    print(f"{title} 的 Top 10词及其出现次数：", top_10_words)

# 分析 fengchuoibanxia 数据集
analyze_word_frequency_top(fengchuoibanxia_df, "风吹半夏")

# 分析 shanhaiqing 数据集
analyze_word_frequency(shanhaiqing_df, "山海情")


风吹半夏 的 Top 10词及其出现次数： [(' ', 219), ('半夏', 89), ('很', 86), ('赵丽颖', 82), ('啊', 71), ('她', 68), ('看', 66), ('太', 63), ('剧', 58), ('他', 58)]
山海情 的 Top 10词及其出现次数： [(' ', 1233), ('啊', 455), ('看', 452), ('太', 356), ('很', 324), ('真的', 314), ('演员', 297), ('这个', 278), ('得', 268), ('水花', 234)]


In [30]:
import pandas as pd
import jieba
from collections import Counter
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import CurrentConfig, NotebookType

# 设置当前配置为Jupyter Notebook
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

# 加载清理后的数据
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_cleaned.xlsx')
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_cleaned.xlsx')

# 自定义停用词列表
stop_words = set([
    '的', '了', '是', '我', '都', '有', '好', '也', '不', '在', 
    '与', '和', '这', '那', '要', '就', '人', '你', '说', '而', '对', '着', '如', '于'
])

# 定义一个函数生成词云图
def generate_wordcloud(df, title):
    # 分词并去除停用词
    df['分词'] = df['Comment'].apply(lambda x: [word for word in jieba.cut(x) if word not in stop_words] if isinstance(x, str) else [])
    
    # 统计词频
    all_words = [word for comment in df['分词'] for word in comment]
    word_counts = Counter(all_words)

    # 准备数据
    words_data = [(word, count) for word, count in word_counts.items()]

    # 创建词云图
    wordcloud = (
        WordCloud()
        .add("", words_data, word_size_range=[20, 100])
        .set_global_opts(title_opts=opts.TitleOpts(title=title))
    )
    
    # 返回词云图对象
    return wordcloud

# 生成风吹半夏的词云图并显示
fengchuoibanxia_wordcloud = generate_wordcloud(fengchuoibanxia_df, "风吹半夏")
fengchuoibanxia_wordcloud.render_notebook()

# 生成山海情的词云图并显示
shanhaiqing_wordcloud = generate_wordcloud(shanhaiqing_df, "山海情")
shanhaiqing_wordcloud.render_notebook()


In [31]:
import pandas as pd
from snownlp import SnowNLP

# 加载清理后的数据
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_cleaned.xlsx')
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_cleaned.xlsx')

# 定义一个函数进行情感分析
def analyze_sentiment(df, title):
    # 情感分析
    df['情感评分'] = df['Comment'].apply(lambda x: SnowNLP(x).sentiments if isinstance(x, str) else None)

    # 分类评论
    df['情感分类'] = df['情感评分'].apply(
        lambda x: '正向' if x > 0.5 else ('负向' if x < 0.5 else '中性') if x is not None else '未知'
    )

    # 统计每种情感评论的数量
    sentiment_counts = df['情感分类'].value_counts()
    print(f"{title} 的情感分类统计：")
    print(sentiment_counts)

    # 打印每个类别的前10个评论
    for sentiment in sentiment_counts.index:
        print(f"\n{title} 的 {sentiment} 评论前10个：")
        print(df[df['情感分类'] == sentiment]['Comment'].head(10).tolist())

# 分析 fengchuoibanxia 数据集情感
analyze_sentiment(fengchuoibanxia_df, "风吹半夏")

# 分析 shanhaiqing 数据集情感
analyze_sentiment(shanhaiqing_df, "山海情")


风吹半夏 的情感分类统计：
情感分类
正向    677
负向    248
中性     22
Name: count, dtype: int64

风吹半夏 的 正向 评论前10个：
['恭喜赵丽颖二封金鹰视后', '恭喜赵丽颖凭借风吹半夏获得飞天奖祝赵丽颖越来越好祝世上的许半夏们事业一路长虹', '听说这部剧上了飞天奖来看看小赵的旧作', '恭喜飞天视后 赵丽颖', '赵丽颖太棒了付出的努力是有收获的', '三个情趣相投之人', '有穎寶必追', '必须支持丽颖的电视剧', '瞎判很明显的正当防卫', '这剧好剧情好演员远超那些什么霸道总裁的烂剧']

风吹半夏 的 负向 评论前10个：
['飞天视后时代来评论', '嘩歐豪是粗線條人配搭趙麗穎新綽頭又是對趙麗穎的一個大挑戰 且看看她的新元素如何', '半夏已露36歲的中年婦人樣貌及體態', '廣告不是一般多', '刚看五分钟 我想说1996年我不信有这样的停车场 以及赵姐的墨镜款式也太超前', '没字幕', '趙麗穎的演技和性格都太招人喜歡了這樣的演員應該走花路', '頭髮擋住眼睛真受不了', '小童啊你是河南老乡是吧', '91年不可能三百多起码几千元']

风吹半夏 的 中性 评论前10个：
['这是欧豪吗', '音乐绝了', '解气', '有点假', '一姐', nan, nan, nan, nan, '1324']
山海情 的情感分类统计：
情感分类
正向    3594
负向    1146
中性      27
Name: count, dtype: int64

山海情 的 正向 评论前10个：
['经典好剧', '方言版真好感谢正午阳光', '黄轩好演员啊', '我又来了', '第二次看山海情', '很好看 但為什麼不是先規劃好下水道 電力水利系統 街道 鋪馬路 招商 再移民 先把人移到沙漠 連土豆都難種阿 領導再想什麼', '张嘉译那个痞子样', '突然想起了三峡移民工程不能适应当地的生活又回不去的家乡', '电视剧演的太温柔了我经历的是强制执行的只不过我们那里环境没有这么夸张哈', '拍的太好了现在的电视剧没法看偶尔给这种剧看太好了']

山海情 的 负向 评论前10个：
['243 非逼着人家去那种前不着村后不着店不鸟不拉屎的地方强迫搬家谁会愿意嘛你县领导怎么不举家带口作表率先去住在那种

In [32]:
import pandas as pd
from snownlp import SnowNLP

# 加载清理后的数据
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_cleaned.xlsx')
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_cleaned.xlsx')

# 定义一个函数进行情感分析
def analyze_sentiment(df, title):
    # 情感分析
    df['情感评分'] = df['Comment'].apply(lambda x: SnowNLP(x).sentiments if isinstance(x, str) else None)

    # 分类评论
    df['情感分类'] = df['情感评分'].apply(
        lambda x: '正向' if x > 0.5 else ('负向' if x < 0.5 else '中性') if x is not None else '未知'
    )

    # 统计每种情感评论的数量
    sentiment_counts = df['情感分类'].value_counts()
    print(f"{title} 的情感分类统计：")
    print(sentiment_counts)

    # 打印每种情感评分最高的前10个评论
    for sentiment in sentiment_counts.index:
        top_comments = df[df['情感分类'] == sentiment].nlargest(10, '情感评分')
        print(f"\n{title} 的 {sentiment} 评论评分最高的前10个评论：")
        for index, row in top_comments.iterrows():
            print(f"评论: {row['Comment']}, 情感评分: {row['情感评分']}")

# 分析 fengchuoibanxia 数据集情感
analyze_sentiment(fengchuoibanxia_df, "风吹半夏")

# 分析 shanhaiqing 数据集情感
analyze_sentiment(shanhaiqing_df, "山海情")


风吹半夏 的情感分类统计：
情感分类
正向    677
负向    248
中性     22
Name: count, dtype: int64

风吹半夏 的 正向 评论评分最高的前10个评论：
评论: 我不是爱追剧的人这类型的剧更不是我喜爱的类型因为喜爱赵丽颖这部剧一次次出现在我的面前可我也一直没有去看终于有了机会能静下心耐心的看了我只想说撇开所有演员精湛的演技导演幕后方方面面的优秀暂且不谈这部剧最后的升华真的是震惊到我了想说点啥感觉语言如此苍白只有情绪翻江倒海感恩有你许半夏感恩有你赵丽颖, 情感评分: 1.0
评论: 精彩的剧每一个细节和场景都是真实的写照感同身受 商场如战场学习到很多的人情世故半夏的整个演出气场和气质都很到位非常大气淡定的面对一切女老板的天花板 如果关于商业的流程和演绎可以更多就好了更多是大起大落的直接展现有些措不及防 总而言之好剧, 情感评分: 0.9999999999999902
评论: 陈晓十年前与赵丽颖 炒了几天cp陈晓觉得不合适强拆了cp自此赵 及其工作团队 用9年时间去侮辱网暴 陈晓导致陈晓近乎抑郁差点毁了陈晓的健康和职业梦想 短暂的喜欢却不可能修成正果因为他们俩为人处事修为相差太远三观严重不一致 赵丽颖打击拉踩别人以抬高自己无论是对自己老公还是合作伙伴甚至闺蜜但凡与她同框或者被她放在同一个文章都是被她拉踩贬低以抬高自己 如此素养深遭国人反感进而更多的遭受网友厌恶 VS 陈晓鼓励欣赏为主正能量满满始终积极向上充满阳光一直是一个通透简单干净纯粹的大男孩与人为善8年前修为已经不一般连当年 年仅10岁的对手滑雪演员 苏翊鸣他在台上媒体面前称呼他为滑雪老师, 情感评分: 0.9999999999999529
评论: 风吹半夏讲述了以许半夏为代表的有志者怀抱雄心壮志在改革开放的大潮中奔流涌动积极探索不断创新的故事展现了上世纪九十年代中小企业在时代浪潮中生存的现实写照赵丽颖太好看了演技没说的一级棒, 情感评分: 0.9999999999876856
评论: 好剧很感动眼泪也流了不少深深的被许半夏的人格魅力所折服希望多出一些这样的好剧支持你赵丽颖, 情感评分: 0.9999999994633753
评论: 这部剧美丽而富有意义感谢你带来这么棒的剧赵丽颖作为许班霞永远在我心中, 情感评分: 0.9999999950046135
评论: 人类社

风吹半夏 的情感分类统计：
情感分类
正向    677
负向    248
中性     22
Name: count, dtype: int64

风吹半夏 的 正向 评论评分最高的前10个评论：
评论: 我不是爱追剧的人这类型的剧更不是我喜爱的类型因为喜爱赵丽颖这部剧一次次出现在我的面前可我也一直没有去看终于有了机会能静下心耐心的看了我只想说撇开所有演员精湛的演技导演幕后方方面面的优秀暂且不谈这部剧最后的升华真的是震惊到我了想说点啥感觉语言如此苍白只有情绪翻江倒海感恩有你许半夏感恩有你赵丽颖, 情感评分: 1.0
评论: 精彩的剧每一个细节和场景都是真实的写照感同身受 商场如战场学习到很多的人情世故半夏的整个演出气场和气质都很到位非常大气淡定的面对一切女老板的天花板 如果关于商业的流程和演绎可以更多就好了更多是大起大落的直接展现有些措不及防 总而言之好剧, 情感评分: 0.9999999999999902
评论: 陈晓十年前与赵丽颖 炒了几天cp陈晓觉得不合适强拆了cp自此赵 及其工作团队 用9年时间去侮辱网暴 陈晓导致陈晓近乎抑郁差点毁了陈晓的健康和职业梦想 短暂的喜欢却不可能修成正果因为他们俩为人处事修为相差太远三观严重不一致 赵丽颖打击拉踩别人以抬高自己无论是对自己老公还是合作伙伴甚至闺蜜但凡与她同框或者被她放在同一个文章都是被她拉踩贬低以抬高自己 如此素养深遭国人反感进而更多的遭受网友厌恶 VS 陈晓鼓励欣赏为主正能量满满始终积极向上充满阳光一直是一个通透简单干净纯粹的大男孩与人为善8年前修为已经不一般连当年 年仅10岁的对手滑雪演员 苏翊鸣他在台上媒体面前称呼他为滑雪老师, 情感评分: 0.9999999999999529
评论: 风吹半夏讲述了以许半夏为代表的有志者怀抱雄心壮志在改革开放的大潮中奔流涌动积极探索不断创新的故事展现了上世纪九十年代中小企业在时代浪潮中生存的现实写照赵丽颖太好看了演技没说的一级棒, 情感评分: 0.9999999999876856
评论: 好剧很感动眼泪也流了不少深深的被许半夏的人格魅力所折服希望多出一些这样的好剧支持你赵丽颖, 情感评分: 0.9999999994633753
评论: 这部剧美丽而富有意义感谢你带来这么棒的剧赵丽颖作为许班霞永远在我心中, 情感评分: 0.9999999950046135
评论: 人类社会资本的走向最高境界 不是金融资本不是官僚资本也不是国有资本而应该是社会资本, 情感评分: 0.9999999930653756
评论: 这部剧太上瘾了我等不及要看后面的剧集了赵丽颖在这里表现得非常棒所有演员都很出色非常真实符合那个时代我记得当时的所有时尚很大的耳环厚底鞋和波浪卷发, 情感评分: 0.9999999921704715
评论: 太美丽也太感人了我看过很多赵丽颖的剧她是一位了不起且出色的女演员我爱她可爱的笑容, 情感评分: 0.9999999644708643
评论: 太感人了见过父辈创业的不易亲历过自己创业的艰辛更能体会创业人的艰难, 情感评分: 0.9999999582681198

风吹半夏 的 负向 评论评分最高的前10个评论：
评论: 1996年的时候手机还叫大哥大吧有谁记得什么时候改叫手机的吗, 情感评分: 0.49906853235525694
评论: 这村长就是犯贱直接走了就行别租了以后事更多, 情感评分: 0.49855305294168717
评论: 感觉伍建设跟裘反过来了伍建设原本应该是遵纪守法的真是被利益冲昏头脑了啥也不顾了, 情感评分: 0.4940960422189554
评论: 原著是女主决定污染滩涂的电视剧为了过审把许半夏改得没那么坏, 情感评分: 0.48760258340502294
评论: 这一集感觉柯蓝的演技有点儿用力过猛尬, 情感评分: 0.48647436391452614
评论: 可能会挨骂可是我还是要说不喜欢童, 情感评分: 0.4856517427227611
评论: 好看坐等更新, 情感评分: 0.4849977554215511
评论: 无法皆空 因果不空, 情感评分: 0.4849256321084442
评论: 东叔 应该不缺钱啊, 情感评分: 0.48476700523456906
评论: 喜欢丽颖又不想看欧豪, 情感评分: 0.48405465616423127

风吹半夏 的 中性 评论评分最高的前10个评论：
评论: 这是欧豪吗, 情感评分: 0.5
评论: 音乐绝了, 情感评分: 0.5
评论: 解气, 情感评分: 0.5
评论: 有点假, 情感评分: 0.5
评论: 一姐, 情感评分: 0.5
评论: 1324, 情感评分: 0.5
评论: L, 情感评分: 0.5
评论: 蓝鸟, 情感评分: 0.5
评论: 张翰, 情感评分: 0.5
评论: nan, 情感评分: nan
山海情 的情感分类统计：
情感分类
正向    3594
负向    1146
中性      27
Name: count, dtype: int64

山海情 的 正向 评论评分最高的前10个评论：
评论: 这部电视剧把中国扶贫工作如此近距离地展现在广大老百姓的面前扶贫政策扶贫干部以及中国贫困地区的人民都这么清晰地鲜活生动地展现在我们面前不断感动着我们这些对农村生活一无所知的国人感谢这些为了脱贫致富而沤心沥血的普通人感谢这些使中国一天天走向繁荣富强的人们加油中国, 情感评分: 1.0
评论: 我是个山东人因为我父亲在我小时候经常出差到西北至新疆地区回到山东就会讲西北地区的人闻趣事和美食使得我从小就有西北情结这部剧已经追到这第十集了特别感动于福建人民对西北环境如此恶劣和在西北吊装的农民一无所有的情况下一点一滴的帮助和关怀多亏国家利用南方沿海地区经济发达的优势给予相关政策展开扶贫和支援西北让西北贫困地区的人民生活水平有了质的飞跃也感叹如今现代这太平年代还会经常看到各地还会有地区 相互歧视的言论他们并不知道有那么个特殊的时期特殊的年代 当时的中国人民是怎样相互帮助和扶持的 希望他们有机会多看看这部剧, 情感评分: 1.0
评论: 一口气看完了真是好剧因为自己也是偏远农村出来的有好多让我能共情的剧情 麦苗去福建工厂上班让我想起了自己曾经在工厂工作的日子虽不是流水线女工但也能深深体会到车间工人的不易 还有女工往家里寄钱大学毕业后开始上班也需要经常给家里寄钱另外白老师努力想留住每一个学生因为他知道教育的重要性上初中那会也经常听老师说去家访要让不去上学的同学继续上学但因为大部分同学是少数民族有的同学初中没有毕业就回家结婚生子了而有的是因为家庭经济情况没有办法只能不上学去打工记得老师说去有的同学家里都没有凳子坐家长给点东西吃都不忍心吃现实逼人那 扶贫的那部分剧情虽然已经离开家乡太多年了但会经常听家里的亲人和朋友提及扶贫 我哥哥就有点德宝的角色做工程的现在也在做一些国家的扶贫项目妹妹类似德福的角色公家单位而我却有点德花的角色读书上学直到大学后面出来留学看着那些艰苦贫困地区的人们都在努力的生活我又有什么理由不好好的活着过好这一生呢, 情感评分: 1.0
评论: 某个瞬间看到了奶奶年轻劳作的身影看到了爷爷下煤矿的辛苦看到了那个时代的不容易看到了父母给孩子带远行行李的那一幕记起了自己离家上学时父母爷奶准备的大把东西自己还在嫌弃放不下这是一部有温度的剧不止让你动容还能让你感受父母爷奶辈的青春奋斗时光, 情感评分: 1.0
评论: 尤勇演的真不错就觉得不是在演戏他就是那样的人农民阶级的善良勤劳质朴和他们的劣根性自私性演绎的淋漓尽致 关于这个劣根性和自私性路遥先生在平凡的世界里双水村实行分产到户村民们分公社的资产的那一段有着深刻生动的描写和分析 注意我可没有贬低瞧不起农民的意思实事求是路遥先生出身地地道道的陕北农家我个人祖上也是来自山东的农民千万别把我想成看不起农民兄弟的人看不起农民的人没有良心, 情感评分: 0.9999999999999998
评论: 凌教授开门看到的情景把我看哭了凌教授真的是非常好非常可爱堂堂科研人员一言不合就动手还把自己折腾骨折了我该说他有辱斯文呢还是心有大爱呢颠覆了我对影视作品中知识分子形象的认知粗鲁冲动起来都这么可爱唯此一人也祖峰演的慈父也非常可爱吃了就不想家了吃了就不想家了他的演技在我心目中是中国第一流的真的演谁是谁黄轩热依扎等等都演得很好开始我是根本不打算看这种题材的剧的看了后发现拍得很实在朴实演员有实力化妆也非常真好剧难得的好剧, 情感评分: 0.9999999999999996
评论: 感谢本电视剧制作者再现了贫穷山区人民脱贫致富真实的艰辛的历程让我们看到了国家的力量社会主义大协作精神的力量日月换新天个人的力量是无论如何都无法实现的故事生动感人众多老戏骨演得精采片尾曲非常好听一部不可多得的好电视剧, 情感评分: 0.9999999999999996
评论: 没想到这么快就大结局了意犹未尽的感觉很不舍得这剧我给满分剧情节奏快不拖沓演员演技到位妆效道具都很真实让人有代入感细节也很用心感谢孔笙导演和制作团队让我们看到这么好的剧给2021年国产电视剧开了个好头希望以后能看到更多制作精良的好剧, 情感评分: 0.9999999999999989
评论: 小时候我们村全是留守儿童包括我很多父母都去广东打工一年回来一次很快同学们初中毕业也纷纷走上了南下打工之路一个班20个人里只有我在内3个人考上了高中又考上了不错的大学彻底走出了山村也许是知识改变命运吧所幸这些年家乡发展快了大家都通过不同的方式经济上富足起来, 情感评分: 0.9999999999999984
评论: 看到各地航班起飞的画面我泪目了太不容易了中国的脱贫工作原来只是听说过脱贫致富这个名词这部剧把这项伟大的事业如此真实地呈现在我们这些普通百姓的面前它不仅仅是捐钱捐物或是政府发放福利那么简单的政府的扶持和各项政策各级扶贫干部专家的无私付出贫困地区的民众积极配合辛勤劳作一个都不能少为祖国这么多优秀平凡的人们点赞, 情感评分: 0.9999999999999984

山海情 的 负向 评论评分最高的前10个评论：
评论: 大有叔演的真是太地道了, 情感评分: 0.49994171917319297
评论: 这些才是为人民办事的好干部, 情感评分: 0.49957898311194593
评论: 水花看的我热泪盈眶真的太吃苦耐劳了, 情感评分: 0.4994636563915813
评论: 从未如此期待过, 情感评分: 0.4992754828759026
评论: 都是实力派啊那个时候的他们太苦了, 情感评分: 0.49887742870874474
评论: 西北漢子西北地貌西北話西北風情非常接地氣的好戲讚, 情感评分: 0.49886879785049476
评论: 姚大嘴heytui, 情感评分: 0.4988130785622946
评论: 方言版比较容易让人飚泪, 情感评分: 0.49820770558657446
评论: 看的流眼泪流鼻涕的还以为我感冒了呢, 情感评分: 0.4979253541467783
评论: 看的人是不是越来越少了怎么评论越来越少了, 情感评分: 0.49664278405933315

山海情 的 中性 评论评分最高的前10个评论：
评论: 二刷, 情感评分: 0.5
评论: 这个祖峰啊, 情感评分: 0.5
评论: 黄轩, 情感评分: 0.5
评论: 水窖是啥, 情感评分: 0.5
评论: 二刷, 情感评分: 0.5
评论: 这真是开荒啊, 情感评分: 0.5
评论: 追剧, 情感评分: 0.5
评论: 莆田, 情感评分: 0.5
评论: No1每天是我, 情感评分: 0.5
评论: 黄轩, 情感评分: 0.5


In [33]:
import pandas as pd
from snownlp import SnowNLP

# 加载清理后的数据
fengchuoibanxia_df = pd.read_excel('fengchuoibanxia_comments_cleaned.xlsx')
shanhaiqing_df = pd.read_excel('shanhaiqing_comments_cleaned.xlsx')

# 定义一个函数进行情感分析
def analyze_sentiment(df, title):
    # 情感分析
    df['情感评分'] = df['Comment'].apply(lambda x: SnowNLP(x).sentiments if isinstance(x, str) else None)

    # 分类评论
    df['情感分类'] = df['情感评分'].apply(
        lambda x: '正向' if x > 0.5 else ('负向' if x < 0.5 else '中性') if x is not None else '未知'
    )

    # 统计每种情感评论的数量
    sentiment_counts = df['情感分类'].value_counts()
    print(f"{title} 的情感分类统计：")
    print(sentiment_counts)

    # 打印每种情感评分最高的前10个评论
    for sentiment in sentiment_counts.index:
        if sentiment == '负向':
            top_comments = df[df['情感分类'] == sentiment].nsmallest(10, '情感评分')  # 负向评论按评分最小排序
        else:
            top_comments = df[df['情感分类'] == sentiment].nlargest(10, '情感评分')  # 正向和中性评论按评分最大排序
        
        print(f"\n{title} 的 {sentiment} 评论评分最高的前10个评论：")
        for index, row in top_comments.iterrows():
            print(f"评论: {row['Comment']}, 情感评分: {row['情感评分']}")

# 分析 fengchuoibanxia 数据集情感
analyze_sentiment(fengchuoibanxia_df, "风吹半夏")

# 分析 shanhaiqing 数据集情感
analyze_sentiment(shanhaiqing_df, "山海情")


风吹半夏 的情感分类统计：
情感分类
正向    677
负向    248
中性     22
Name: count, dtype: int64

风吹半夏 的 正向 评论评分最高的前10个评论：
评论: 我不是爱追剧的人这类型的剧更不是我喜爱的类型因为喜爱赵丽颖这部剧一次次出现在我的面前可我也一直没有去看终于有了机会能静下心耐心的看了我只想说撇开所有演员精湛的演技导演幕后方方面面的优秀暂且不谈这部剧最后的升华真的是震惊到我了想说点啥感觉语言如此苍白只有情绪翻江倒海感恩有你许半夏感恩有你赵丽颖, 情感评分: 1.0
评论: 精彩的剧每一个细节和场景都是真实的写照感同身受 商场如战场学习到很多的人情世故半夏的整个演出气场和气质都很到位非常大气淡定的面对一切女老板的天花板 如果关于商业的流程和演绎可以更多就好了更多是大起大落的直接展现有些措不及防 总而言之好剧, 情感评分: 0.9999999999999902
评论: 陈晓十年前与赵丽颖 炒了几天cp陈晓觉得不合适强拆了cp自此赵 及其工作团队 用9年时间去侮辱网暴 陈晓导致陈晓近乎抑郁差点毁了陈晓的健康和职业梦想 短暂的喜欢却不可能修成正果因为他们俩为人处事修为相差太远三观严重不一致 赵丽颖打击拉踩别人以抬高自己无论是对自己老公还是合作伙伴甚至闺蜜但凡与她同框或者被她放在同一个文章都是被她拉踩贬低以抬高自己 如此素养深遭国人反感进而更多的遭受网友厌恶 VS 陈晓鼓励欣赏为主正能量满满始终积极向上充满阳光一直是一个通透简单干净纯粹的大男孩与人为善8年前修为已经不一般连当年 年仅10岁的对手滑雪演员 苏翊鸣他在台上媒体面前称呼他为滑雪老师, 情感评分: 0.9999999999999529
评论: 风吹半夏讲述了以许半夏为代表的有志者怀抱雄心壮志在改革开放的大潮中奔流涌动积极探索不断创新的故事展现了上世纪九十年代中小企业在时代浪潮中生存的现实写照赵丽颖太好看了演技没说的一级棒, 情感评分: 0.9999999999876856
评论: 好剧很感动眼泪也流了不少深深的被许半夏的人格魅力所折服希望多出一些这样的好剧支持你赵丽颖, 情感评分: 0.9999999994633753
评论: 这部剧美丽而富有意义感谢你带来这么棒的剧赵丽颖作为许班霞永远在我心中, 情感评分: 0.9999999950046135
评论: 人类社